In [1]:
import os
from datetime import datetime
from datetime import date
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import excelCol
import matplotlib.pyplot as plt
%matplotlib inline

#### Input Data for Program


In [2]:
labels = ['AAPL', 'FB', 'MSFT']
start_date = '2013-2-01'
end_date = date.today()

In [3]:
#Get Data for tickers
ticker_info ={}
for i, j in zip(labels, range(len(labels))):
       ticker_info[j] = web.DataReader(i, "av-daily-adjusted", start=datetime(2017, 2, 9),end=datetime(2020, 3, 31),api_key='64WDJQ42PC1EKNEF')

    

In [96]:
# Merge Data frames with only adjusted close price for tickers
df = pd.DataFrame(ticker_info[0]['adjusted close'])
for key in ticker_info.keys():
    if key != 0:
        df = df.merge(ticker_info[key]['adjusted close'], left_index = True, right_index = True) 


In [97]:
# Assign the columns names to ticker names
df.columns = [labels]
df.index = pd.to_datetime(df.index)

In [98]:
df = df.resample('M').ffill().pct_change()
df.dropna(inplace = True)


In [99]:
three_yr_ret = np.power(np.prod(1 + df.tail(36)), 12/36) -1
five_yr_ret = np.power(np.prod(1 + df.tail(60)), 12/60) -1
one_yr_ret = np.power(np.prod(1 + df.tail(12)), 12/12) -1

In [100]:
#df.index= df.index - pd.datetime(1899, 12, 31)

In [101]:
df.head()

,AAPL,FB,MSFT
2017-03-31,0.048690,0.048030,0.029385
2017-04-30,-0.000070,0.057726,0.039477
2017-05-31,0.067770,0.008053,0.025890
2017-06-30,-0.057214,-0.003169,-0.013029
2017-07-31,0.032704,0.121009,0.054692


In [115]:
writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter',date_format='mmm dd yyyy',datetime_format='mm/dd/yyyy',)
startrow = 5
startcol = 3

df.to_excel(writer, sheet_name='Sheet1', startrow= 5, startcol=3, index = True)
# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

format1 = workbook.add_format({'num_format': '#,##0.00%'})
format2 = workbook.add_format({'num_format': '0%'})
dateformat = workbook.add_format({'num_format': 'mm-dd-yyyy'})

worksheet.set_column(first_col=0, last_col=10, cell_format=format1)
#worksheet.set_column(first_col=3, last_col=3, cell_format=dateformat)

chart1 = workbook.add_chart({'type': 'line'})  
for i in range(len(df.columns)):
    col = i + startcol
    chart1.add_series({
        'name':       ['Sheet1', startrow, col + 1],
        'categories': ['Sheet1', startrow, startcol, startrow + len(df), startcol],
        'values':     ['Sheet1', startrow + 1, col + 1, startrow + len(df), col + 1 ],
        'data_labels': { 'position': 'below'},
    })

# Add a chart title   
chart1.set_title ({'name': 'Return Data'})  
    
# Add x-axis label  

chart1.set_x_axis({'name': 'Date','num_format': 'mmm-yyyy', 'label_position': 'low'})

# Add y-axis label  
chart1.set_y_axis({'name': 'Return'})  
    
# Set an Excel chart style.  
chart1.set_style(11)  
    
# add chart to the worksheet with given 
# offset values at the top-left corner of 
# a chart is anchored to cell D2 .   
worksheet.insert_chart('K2', chart1, {'x_offset': 25, 'y_offset': 10})  
    
    
writer.save()